# Introduction to Bedrock - Text Processing Sample : Understanding a Contact Center servicing request

--- 

In this demo notebook, we demonstrate how to use the Bedrock Python SDK for a text processing example. We show how to use Bedrock Foundational Models derive key information from a contact center transcript. 

---

Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 2.0) kernel.

1. [Set Up](#1.-Set-Up-and-API-walkthrough)
2. [Transcript Processing](#2.-Transcript-Processing)

## 1. Set Up and API walkthrough

---
Before executing the notebook for the first time, execute this cell to add bedrock extensions to the Python boto3 SDK

---

**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [1]:
bedrock_region = "us-west-2" 

In [2]:
if bedrock_region == "us-east-1":    
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://bedrock.us-east-1.amazonaws.com"
    }
elif bedrock_region == "us-west-2":  
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://prod.us-west-2.frontend.bedrock.aws.dev"
    }

#### Now let's set up our connection to the Amazon Bedrock SDK using Boto3

In [3]:
import boto3
import json
bedrock = boto3.client(
    service_name='bedrock',
    region_name=bedrock_config["region_name"],
    endpoint_url=bedrock_config["endpoint_url"]
)

#### We can validate our connection by testing out the _list_foundation_models()_ method, which will tell us all the models available for us to use 

In [4]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '17480e3c-8881-48c5-8e72-3ebf23342399',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 14 Jul 2023 05:00:13 GMT',
   'content-type': 'application/json',
   'content-length': '861',
   'connection': 'keep-alive',
   'x-amzn-requestid': '17480e3c-8881-48c5-8e72-3ebf23342399'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/stability.stable-diffusion-xl',
   'modelId': 'stability.stable-diffusion-xl'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/ai21.j2-grande-instruct',
   'modelId': 'ai21.j2-grande-instruct'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/ai21.j2-jumbo-instruct',
   'modelId': 'ai21.j2-jumbo-in

#### In this Notebook we will be using the invoke_model() method of Amazon Bedrock. This will be the primary method we use for most of our Text Generation and Processing tasks. 

##### The mandatory parameters required to use this method are, where _modelId_ represents the Amazon Bedrock model ARN, and _body_ which is the prompt for our task. The _body_ prompt will change depending on the foundational model provider selected. We walk through this in detail below

```
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

## 2. Transcript Processing

#### Let's now try out the Amazon Bedrock models to have it answer key questions about a contact center transcript

In [5]:
prompt_data ="""Below is a contact center transcription Start of Transcript Terry: Hi there this is Any Company Insurance, my name is Terry and I am the claims expert that will be helping you, who do I have the pleasure of speaking with today Joan: Hello, this is Joan Richards Terry: Hi Joan how can I help you today? Joan: I wanted to check up on the status of my claim cheque, it's been 4 weeks and I haven't recieved that in the mail. Terry: I'm so sorry to hear that Joan, we will fix this right up, do you have a claim number? Joan: Yeah give me a second Terry: Take your time Joan: CM-5343-434934 Terry: Great I'll repeat that back to you: CM-5343-434934 Joan: Thats correct Terry: So it seems like we did send it, but it wasn't cashed in. We can send another one, but we will need to confirm some information. Can you confirm your Date of Birth and an address that we might have on file? Joan: March 15 1978, 452 East 1st Street, Redmond, Washington Terry: Can I have your postal code? Joan: 98052 Terry: Great well Joan all that looks good on my end, the team will send another cheque in the mail. I have created a tracking number that I can share when you're ready. Joan: Ready Terry: USPS-53434343 Terry: Awesome Joan is there anything else I can help with today? Joan: I'm good thank you. Terry: Great have a wonderful day! End of Transcript Answer the following Question Why did the customer call into the contact center? What was the claim number they asked about? What was the customers address?""" #If you'd like to try your own prompt, edit this parameter!

### Let's start by using the Amazon Titan Large Model

In [6]:
body = json.dumps(
    {
        "inputText": prompt_data,
        "textGenerationConfig":
        {
            "maxTokenCount":4096,
            "stopSequences":[],
            "temperature":0,
            "topP":0.9
        }
    }
)
modelId = 'amazon.titan-tg1-large' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('results')[0].get('outputText'))


Joan called in to check the status of her claim cheque, that hasn't come in the mail yet. The claim number she provided was CM-5343-434934. The customer provided her date of birth and address to confirm on the file.


### Let's try the prompt with the Anthropic Claude Instant Model on Bedrock

In [7]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 50})
modelId = 'anthropic.claude-instant-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))

AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeModel operation: Your account is not authorized to invoke this API operation.

### Let's try the prompt with the Jurrasic Grande Model on Bedrock

In [ ]:
body = json.dumps({"prompt": prompt_data})
modelId = 'ai21.j2-grande-instruct' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completions')[0].get('data').get('text'))